# 이상신호 감지 기반 비정상 작동 진단 - EDA

## 목표
- Macro-F1 Score > 0.9 달성
- 데이터 특성 파악 및 전처리 전략 수립

## 평가 지표
- **Macro-F1 Score**: 각 클래스의 F1 Score를 평균낸 값
- 클래스 불균형에 민감하므로 모든 클래스를 균등하게 잘 예측해야 함

In [ ]:
# 필수 라이브러리 임포트
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import f1_score, classification_report, confusion_matrix
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8-darkgrid')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

print('라이브러리 로드 완료')

## 1. 데이터 로드 및 기본 정보 확인

In [ ]:
# 데이터 로드
train = pd.read_csv('../open/train.csv')
test = pd.read_csv('../open/test.csv')
submission = pd.read_csv('../open/sample_submission.csv')

print(f"Train shape: {train.shape}")
print(f"Test shape: {test.shape}")
print(f"Submission shape: {submission.shape}")
print(f"\n피처 수: {train.shape[1] - 2} (ID, target 제외)")
print(f"Train 샘플 수: {train.shape[0]:,}")
print(f"Test 샘플 수: {test.shape[0]:,}")

In [ ]:
# 데이터 미리보기
print("Train 데이터 샘플 (처음 5개):")
display(train.head())

print("\n컬럼 리스트:")
print(train.columns.tolist())

In [ ]:
# 데이터 타입 확인
print("데이터 타입:")
print(train.dtypes.value_counts())

# 메모리 사용량
print(f"\n메모리 사용량:")
print(f"Train: {train.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
print(f"Test: {test.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

## 2. Target 분포 분석 (중요!)

In [ ]:
# Target 분포 확인
target_counts = train['target'].value_counts()
target_ratio = train['target'].value_counts(normalize=True)

print("Target 클래스 분포:")
print("="*50)
for cls in sorted(target_counts.index):
    print(f"Class {cls}: {target_counts[cls]:6d} ({target_ratio[cls]*100:5.2f}%)")
print("="*50)
print(f"총 클래스 수: {train['target'].nunique()}")
print(f"\n클래스 불균형 비율 (최대/최소): {target_counts.max() / target_counts.min():.2f}")

In [ ]:
# Target 분포 시각화
fig = make_subplots(rows=1, cols=2, 
                    subplot_titles=['Target 클래스 분포', 'Target 클래스 비율'],
                    specs=[[{'type': 'bar'}, {'type': 'pie'}]])

# 막대 차트
fig.add_trace(
    go.Bar(x=target_counts.index, y=target_counts.values, 
           text=target_counts.values, textposition='auto',
           name='Count'),
    row=1, col=1
)

# 파이 차트
fig.add_trace(
    go.Pie(labels=target_counts.index, values=target_counts.values,
           textinfo='label+percent', hole=0.3),
    row=1, col=2
)

fig.update_layout(height=400, title_text="Target 클래스 분포 분석")
fig.show()

## 3. 결측치 분석

In [ ]:
# 결측치 확인
train_nulls = train.isnull().sum()
test_nulls = test.isnull().sum()

print("Train 결측치:")
print(f"결측치가 있는 컬럼 수: {(train_nulls > 0).sum()}")
if train_nulls.sum() > 0:
    print(train_nulls[train_nulls > 0])
else:
    print("결측치 없음")

print("\nTest 결측치:")
print(f"결측치가 있는 컬럼 수: {(test_nulls > 0).sum()}")
if test_nulls.sum() > 0:
    print(test_nulls[test_nulls > 0])
else:
    print("결측치 없음")

## 4. 피처 분포 분석

In [ ]:
# 피처 기본 통계
feature_cols = [col for col in train.columns if col not in ['ID', 'target']]
print(f"분석할 피처 수: {len(feature_cols)}")

# 기술 통계
train_stats = train[feature_cols].describe()
print("\n기술 통계 (일부):")
display(train_stats.iloc[:, :10])  # 처음 10개 피처만 표시

In [ ]:
# 각 피처의 분포 특성 분석
feature_info = pd.DataFrame()
feature_info['feature'] = feature_cols
feature_info['mean'] = train[feature_cols].mean().values
feature_info['std'] = train[feature_cols].std().values
feature_info['min'] = train[feature_cols].min().values
feature_info['max'] = train[feature_cols].max().values
feature_info['nunique'] = train[feature_cols].nunique().values
feature_info['skew'] = train[feature_cols].skew().values
feature_info['kurt'] = train[feature_cols].kurtosis().values

# 스케일 차이 분석
feature_info['scale_ratio'] = feature_info['max'] - feature_info['min']

print("피처별 특성 요약:")
display(feature_info.head(10))

print(f"\n유니크 값이 적은 피처 (이진 또는 범주형 가능성):")
low_unique = feature_info[feature_info['nunique'] <= 10]
print(f"개수: {len(low_unique)}")
if len(low_unique) > 0:
    print(low_unique[['feature', 'nunique']].to_string())

In [ ]:
# 피처 분포 히스토그램 (샘플)
sample_features = feature_cols[:12]  # 처음 12개 피처만

fig, axes = plt.subplots(3, 4, figsize=(16, 10))
axes = axes.flatten()

for idx, col in enumerate(sample_features):
    train[col].hist(bins=50, ax=axes[idx], edgecolor='black', alpha=0.7)
    axes[idx].set_title(f'{col}\n(unique: {train[col].nunique()})', fontsize=10)
    axes[idx].set_xlabel('')
    axes[idx].set_ylabel('Frequency')

plt.suptitle('피처 분포 샘플 (처음 12개)', fontsize=14)
plt.tight_layout()
plt.show()

## 5. 타겟별 피처 분포 차이 분석

In [ ]:
# 타겟별 피처 평균값 차이 분석
target_means = train.groupby('target')[feature_cols].mean()

# 각 피처별로 타겟 간 차이가 큰지 분석
feature_importance = pd.DataFrame()
feature_importance['feature'] = feature_cols

# 타겟별 평균의 표준편차 (타겟 간 차이가 클수록 높음)
feature_importance['target_std'] = target_means.std().values

# 전체 표준편차 대비 타겟 간 표준편차 비율
overall_std = train[feature_cols].std()
feature_importance['importance_ratio'] = feature_importance['target_std'] / overall_std.values

# 중요도 순으로 정렬
feature_importance = feature_importance.sort_values('importance_ratio', ascending=False)

print("타겟 구분력이 높은 상위 15개 피처:")
display(feature_importance.head(15))

In [ ]:
# 상위 중요 피처의 타겟별 분포 시각화
top_features = feature_importance.head(6)['feature'].values

fig, axes = plt.subplots(2, 3, figsize=(15, 8))
axes = axes.flatten()

for idx, col in enumerate(top_features):
    for target_val in sorted(train['target'].unique()):
        subset = train[train['target'] == target_val][col]
        axes[idx].hist(subset, bins=30, alpha=0.5, label=f'Target {target_val}', density=True)
    
    axes[idx].set_title(f'{col}', fontsize=10)
    axes[idx].set_xlabel('Value')
    axes[idx].set_ylabel('Density')
    axes[idx].legend(fontsize=8)

plt.suptitle('타겟별 피처 분포 (상위 6개 중요 피처)', fontsize=14)
plt.tight_layout()
plt.show()

## 6. 상관관계 분석

In [ ]:
# 피처 간 상관관계
correlation_matrix = train[feature_cols].corr()

# 높은 상관관계 쌍 찾기
high_corr_pairs = []
for i in range(len(correlation_matrix.columns)):
    for j in range(i+1, len(correlation_matrix.columns)):
        if abs(correlation_matrix.iloc[i, j]) > 0.8:
            high_corr_pairs.append({
                'feature1': correlation_matrix.columns[i],
                'feature2': correlation_matrix.columns[j],
                'correlation': correlation_matrix.iloc[i, j]
            })

if high_corr_pairs:
    high_corr_df = pd.DataFrame(high_corr_pairs).sort_values('correlation', ascending=False)
    print(f"높은 상관관계 (|r| > 0.8) 쌍 수: {len(high_corr_df)}")
    print("\n상위 10개:")
    display(high_corr_df.head(10))
else:
    print("높은 상관관계 (|r| > 0.8) 쌍이 없습니다.")

In [ ]:
# 타겟과 피처 간 상관관계 (선형 관계만 파악 가능)
target_corr = train[feature_cols + ['target']].corr()['target'].drop('target')
target_corr = target_corr.sort_values(key=abs, ascending=False)

print("타겟과 상관관계가 높은 상위 15개 피처:")
print(target_corr.head(15))

# 시각화
plt.figure(figsize=(10, 6))
target_corr.head(20).plot(kind='barh')
plt.xlabel('Correlation with Target')
plt.title('Top 20 Features by Correlation with Target')
plt.tight_layout()
plt.show()

## 7. 이상치 탐지

In [ ]:
# IQR 방법으로 이상치 탐지
def count_outliers_iqr(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    outliers = df[(df[column] < lower_bound) | (df[column] > upper_bound)]
    return len(outliers)

outlier_counts = {}
for col in feature_cols:
    outlier_counts[col] = count_outliers_iqr(train, col)

outlier_df = pd.DataFrame(list(outlier_counts.items()), columns=['feature', 'outlier_count'])
outlier_df['outlier_ratio'] = outlier_df['outlier_count'] / len(train)
outlier_df = outlier_df.sort_values('outlier_ratio', ascending=False)

print("이상치가 많은 상위 10개 피처:")
display(outlier_df.head(10))

## 8. Train/Test 분포 비교

In [ ]:
# Train과 Test의 피처 분포 차이 확인
distribution_diff = pd.DataFrame()
distribution_diff['feature'] = feature_cols
distribution_diff['train_mean'] = train[feature_cols].mean().values
distribution_diff['test_mean'] = test[feature_cols].mean().values
distribution_diff['mean_diff'] = abs(distribution_diff['train_mean'] - distribution_diff['test_mean'])
distribution_diff['train_std'] = train[feature_cols].std().values
distribution_diff['test_std'] = test[feature_cols].std().values
distribution_diff['std_diff'] = abs(distribution_diff['train_std'] - distribution_diff['test_std'])

# 정규화된 차이
distribution_diff['normalized_diff'] = distribution_diff['mean_diff'] / (distribution_diff['train_std'] + 1e-10)

distribution_diff = distribution_diff.sort_values('normalized_diff', ascending=False)

print("Train/Test 분포 차이가 큰 상위 10개 피처:")
display(distribution_diff.head(10))

if distribution_diff['normalized_diff'].max() > 0.5:
    print("\n⚠️ 경고: 일부 피처에서 Train/Test 분포 차이가 큽니다!")
else:
    print("\n✅ Train/Test 분포가 전반적으로 유사합니다.")

## 9. 핵심 인사이트 및 전략

In [ ]:
# 분석 결과 요약
print("="*60)
print("📊 데이터 분석 요약")
print("="*60)

print(f"\n1. 데이터 규모:")
print(f"   - Train: {train.shape[0]:,} samples")
print(f"   - Test: {test.shape[0]:,} samples")
print(f"   - Features: {len(feature_cols)} (X_01 ~ X_52)")
print(f"   - Target Classes: {train['target'].nunique()}")

print(f"\n2. Target 분포:")
print(f"   - 클래스 불균형 비율: {target_counts.max() / target_counts.min():.2f}")
if target_counts.max() / target_counts.min() > 3:
    print(f"   - ⚠️ 클래스 불균형 처리 필요")

print(f"\n3. 결측치:")
print(f"   - Train: {train_nulls.sum()} 개")
print(f"   - Test: {test_nulls.sum()} 개")

print(f"\n4. 피처 특성:")
print(f"   - 낮은 unique 값 피처: {len(low_unique)} 개")
print(f"   - 높은 상관관계 쌍: {len(high_corr_pairs) if 'high_corr_pairs' in locals() else 0} 개")

print("\n" + "="*60)
print("🎯 모델링 전략")
print("="*60)

print("""
1. 데이터 전처리:
   - StandardScaler로 스케일링 (피처 간 스케일 차이 존재)
   - 이상치 처리 고려 (특정 피처에 이상치 다수)

2. Feature Engineering:
   - 피처 간 상호작용 특징 생성
   - 통계적 특징 추가 (mean, std, max-min 등)
   - PCA 또는 차원 축소 고려

3. 모델 선택:
   - Tree 기반 모델 (XGBoost, LightGBM, CatBoost)
   - 앙상블 방법 적용
   - Macro-F1 최적화를 위한 클래스 가중치 조정

4. 검증 전략:
   - Stratified K-Fold (K=5)
   - Macro-F1 Score 모니터링
   - 각 클래스별 성능 균등화

목표: Macro-F1 Score > 0.9
""")

In [ ]:
# 데이터 저장 (전처리용)
train.to_csv('../data/train.csv', index=False)
test.to_csv('../data/test.csv', index=False)
submission.to_csv('../data/sample_submission.csv', index=False)

print("✅ 데이터를 data 폴더에 저장했습니다.")
print("\n다음 단계: Baseline 모델 구축")